# GridSearchCVによるパラメタチューニング
Boston house-pricesデータセットを題材にElasticNetのパラメタをGridSearchCVでチューニング

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Boston house-pricesのロード
boston = load_boston()

X = boston.data
y = boston.target

In [ ]:
# 学習データと評価データに分割（分割比率8:2）
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# デフォルトパラメタ
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)

# 評価データでのR2スコアを算出
pred = elastic_net.predict(X_test)
score = r2_score(y_test, pred)

print('ElasticNet(default) R2 score: %.3f' % score)

In [ ]:
# チューニングモデル
estimator = ElasticNet()

# モデルのパラメタ候補を定義
alpha_params = np.logspace(-3, 3, 7)
l1_ratio_params = np.arange(0.1, 1.0, 0.1)

paramters = {
    'alpha': alpha_params,
    'l1_ratio': l1_ratio_params
}

# パラメタチューニング
model_tuning = GridSearchCV(
    estimator = estimator,     # 識別器
    param_grid = paramters,    # パラメタ
    refit = True,              # bestモデルを学習データ全体で再学習
    cv = 5,                    # Cross validationの分割数
    n_jobs = -1,               # 並列実行数(-1: コア数で並列実行)
    verbose = 0                # メッセージ出力レベル
)

model_tuning.fit(X_train, y_train)

In [ ]:
def PlotGridSearchScores(model_tuning, x_param, line_param):
    x_values = model_tuning.cv_results_['param_' + x_param].data
    x_labels = np.sort(np.unique(x_values))
    x_keys = ['{0:9.2e}'.format(x) for x in x_labels]

    line_values = model_tuning.cv_results_['param_' + line_param].data
    line_labels = np.sort(np.unique(line_values))
    line_keys = ['{0:9.2e}'.format(v) for v in line_labels]

    score = {}

    # (line_key, x_key) -> mean_test_scoreを生成
    for i, test_score in enumerate(model_tuning.cv_results_['mean_test_score']):
        x = x_values[i]
        line_value = line_values[i]

        x_key = '{0:9.2e}'.format(x)
        line_key = '{0:9.2e}'.format(line_value)

        score[line_key, x_key] = test_score

    _, ax = plt.subplots(1,1)
    
    # x_paramをx軸、line_paramを折れ線グラフで表現
    for line_key in line_keys:
        line_score = [score[line_key, x_key] for x_key in x_keys]
        ax.plot(x_labels, line_score, '-o', label=line_param + ': ' + line_key)

    ax.set_title("Grid Search R2 Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(x_param, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95, 0.5, .100), fontsize=15)
    ax.grid('on')

In [ ]:
# チューニング結果を描画
PlotGridSearchScores(model_tuning, 'l1_ratio', 'alpha')

In [ ]:
# チューニングしたモデルを評価
elastic_net_tuned = model_tuning.best_estimator_

# 評価データでのR2スコアを算出
pred = elastic_net_tuned.predict(X_test)
score = r2_score(y_test, pred)

print('ElasticNet(tuned) R2 score: %.3f' % score)